In [8]:
# Import neccessary module
import time
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.datasets import mnist
import timeit
#Tắt ipynb warning
import warnings
warnings.filterwarnings('ignore')

In [9]:
#Ten fold split for validation
def split_list(a_list):
    ten_fold = len(a_list)/10
    return a_list[ten_fold:], a_list[:ten_fold]

In [10]:
#History list for lost (incorrect) sample
class LossHistory(cb.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        batch_loss = logs.get('loss')
        self.losses.append(batch_loss)

In [11]:
#Load mnist.pkl.gz data (handwritting digit)
def load_data(data):
    print ('Loading data...')
    (X_train, y_train), (X_test, y_test) = mnist.load_data(path = data)
    X_train, X_val = split_list(X_train)
    y_train, y_val = split_list(y_train)
    print(len(X_train))
    print(len(X_test))
    print(len(X_val))
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    
    X_train /= 255
    X_test /= 255
    X_val /= 255
    
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    y_val = np_utils.to_categorical(y_val, 10)

    X_train = np.reshape(X_train, (54000, 784))
    X_test = np.reshape(X_test, (10000, 784))
    X_val = np.reshape(X_val, (6000, 784))
    
    print ('Data loaded.')
    return [X_train, X_test, X_val, y_train, y_test, y_val]

In [12]:
#Init Feed Forward Neural Network model use Keras
#Some hyperparameter: Layer 1: 500 nodes, layer 2: 300 nodes, layer 3: 10 nodes (with ten case of classify)        
def init_model():
    start_time = time.time()
    print ('Compiling Model ... ')
    model = Sequential()
    model.add(Dense(500, input_dim=784))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(300))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    rms = RMSprop()
    model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
    print ('Model compiled in {0} seconds'.format(time.time() - start_time))
    return model

In [18]:
#Run model (include: fit model, test accuracy)
#Some hyperparameter: epochs=20, batch=256
def run_network(data=None, model=None, epochs=20, batch=256):
    try:
        start_time = time.time()
        if data is None:
            X_train, X_test, X_val, y_train, y_test, y_val = load_data('mnist.pkl.gz')
        else:
            X_train, X_test, X_val, y_train, y_test, y_val = data

        if model is None:
            model = init_model()

        history = LossHistory()

        print ('Training model...')
        model.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch,
                  callbacks=[history],
                  validation_data=(X_val, y_val), verbose=2)

        print ("Training duration : {0}".format(time.time() - start_time))
        print ('Testing...')
        score = model.evaluate(X_test, y_test, batch_size=16)
        print ("\nNetwork's test score [loss, accuracy]: {0}".format(score))
    except KeyboardInterrupt:
        print (' KeyboardInterrupt')

In [19]:
#Start all
run_network()

Loading data...
54000
10000
6000
Data loaded.
Compiling Model ... 
Model compield in 0.1308126449584961 seconds
Training model...
Train on 54000 samples, validate on 6000 samples
Epoch 1/20
8s - loss: 0.3766 - acc: 0.8842 - val_loss: 0.1360 - val_acc: 0.9615
Epoch 2/20
8s - loss: 0.1637 - acc: 0.9505 - val_loss: 0.1019 - val_acc: 0.9688
Epoch 3/20
8s - loss: 0.1196 - acc: 0.9643 - val_loss: 0.0871 - val_acc: 0.9738
Epoch 4/20
7s - loss: 0.0948 - acc: 0.9701 - val_loss: 0.0726 - val_acc: 0.9792
Epoch 5/20
7s - loss: 0.0831 - acc: 0.9750 - val_loss: 0.0754 - val_acc: 0.9785
Epoch 6/20
7s - loss: 0.0723 - acc: 0.9782 - val_loss: 0.0684 - val_acc: 0.9797
Epoch 7/20
7s - loss: 0.0663 - acc: 0.9804 - val_loss: 0.0694 - val_acc: 0.9810
Epoch 8/20
8s - loss: 0.0589 - acc: 0.9820 - val_loss: 0.0729 - val_acc: 0.9803
Epoch 9/20
8s - loss: 0.0565 - acc: 0.9835 - val_loss: 0.0757 - val_acc: 0.9790
Epoch 10/20
8s - loss: 0.0505 - acc: 0.9848 - val_loss: 0.0645 - val_acc: 0.9835
Epoch 11/20
8s - los